In [7]:
import cv2
import numpy as np

# 读取图像
image = cv2.imread(r"C:\Users\28489\Desktop\paired\3\3.jpg")

# 转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 二值化处理
_, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

# 查找轮廓
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 找到最大的轮廓（假设残片是最大的对象）
largest_contour = max(contours, key=cv2.contourArea)

# 计算质心
M = cv2.moments(largest_contour)
if M["m00"] != 0:
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
else:
    cX, cY = 0, 0

# 在图像上绘制质心
cv2.circle(image, (cX, cY), 5, (0, 0, 255), -1)
cv2.putText(image, f"Centroid: ({cX}, {cY})", (cX - 25, cY - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# 创建一个与原图像大小相同的掩码
mask = np.zeros_like(gray)
cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

# 应用掩码到原图像
result = cv2.bitwise_and(image, image, mask=mask)

# 裁剪图像
x, y, w, h = cv2.boundingRect(largest_contour)
cropped_result = result[y:y + h, x:x + w]

# 计算裁剪后图像中的质心位置
centroid_cropped_x = cX - x
centroid_cropped_y = cY - y

# 在裁剪后的图像上绘制质心
cv2.circle(cropped_result, (centroid_cropped_x, centroid_cropped_y), 5, (0, 0, 255), -1)
cv2.putText(cropped_result, f"Centroid: ({centroid_cropped_x}, {centroid_cropped_y})", (centroid_cropped_x - 25, centroid_cropped_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# 绘制坐标轴
height, width = cropped_result.shape[:2]
# 绘制 x 轴
cv2.line(cropped_result, (0, centroid_cropped_y), (width, centroid_cropped_y), (255, 0, 0), 2)
# 绘制 y 轴
cv2.line(cropped_result, (centroid_cropped_x, 0), (centroid_cropped_x, height), (0, 255, 0), 2)

# 调整窗口大小
scale_percent = 25  # 调整比例，可以根据需要修改
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

# 调整图像大小
resized_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
resized_cropped_result = cv2.resize(cropped_result, dim, interpolation = cv2.INTER_AREA)

# 提取图像轮廓
contour_image = cv2.cvtColor(gray.copy(), cv2.COLOR_GRAY2BGR)
cv2.drawContours(contour_image, [largest_contour], -1, (0, 255, 0), 2)
resized_contour = cv2.resize(contour_image, dim, interpolation = cv2.INTER_AREA)

# 显示结果
cv2.imshow('Original Image', resized_image)
cv2.imshow('Cropped Result', resized_cropped_result)
cv2.imshow('Extracted Contour', resized_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 输出质心坐标
print(f'Centroid: ({cX}, {cY})')


Centroid: (1471, 1191)
